In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

First lets load our data

In [2]:
data = pd.read_csv("./data/iris.csv")
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


We will represent species (or classes) as integers instead of their names. To do this we will use sci-kit's LabelEncoder

In [3]:
label_enc = LabelEncoder()
data["species"] = label_enc.fit_transform(data["species"])

In [4]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


There are multiple variations of Naive Bayes Classifier such as Multinomial Naive Bayes Classifier or Gaussian Naive Bayes Classifier. Generally Naive Bayes Classifier name is used for mentioning Multinomial Naive Bayes Classifier. So in this tutorial we actually implement Multinomial Bayes but for short we will use the name Naive Bayes as it is generally. Naive Bayes uses categorical input. However we have ordinal data. To turn our ordinal data to categorical we will use sci-kit's Discretizer. We will uniformly divide our data into 3 bins. You can try different paramters such as changing the number of bins or using other methods of forming the bins such as k-means. 

In [5]:
from sklearn.preprocessing import KBinsDiscretizer
X = data.iloc[:,:-1]
est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform') # for k-means change strategy to kmeans
est.fit(X)

Xt = est.transform(X)

Xt = Xt.astype(np.int)
data.iloc[:,:-1] = Xt
y = data.iloc[:,-1]
X = data.iloc[:,:-1]

In [6]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,2,0,0,0


Lets check how many classes we have and number of samples belonging to each class. As you can see we have a uniformly distributed data. Each class has 50 samples

In [7]:
print(np.unique(data["species"],return_counts=True))

(array([0, 1, 2]), array([50, 50, 50]))


The bayes theorem is: P(A|B) = P(B|A) * P(A) / P(B). Here P(A|B) is the probabilty of A happening given that B is true. Naive Bayes Classifier is build on this theorem. As an example lets say that A is our label and B is our features. We want to know given our features B what is the probabilty of our sample being label A. To find this we can use bayes theorem. P(A) is just the probabilty of our sample being class A. We can easily compute this from our  dataset. It will be just the frequency of samples with label A. P(B|A) is given that our label is A what is the probabily of our features being the value B. Again using our dataset we just have to compute frequency of the samples with label A and having exact feature value B. 

However doing this is computationaly inefficient. As the number of features and values our features can take grows the number of computations we have to do also grows larger because we have to count sampels of each case. Why this algorithm is called "naive" bayes is because we make an assumption that our features are indepedent. This means that we assume that the probabilty of feature #1 being a given value is not tied to feature #2 any how. With this assumption we don't have to compute frequincies of each case. We will just compute P(B|A) as P(B1,B2,B3|A) = P(B1|A) * P(B2|A) * P(B3|A) (Here B1,B2 and B3 are features). So we will just look at the probabilty of each feature being a given value and multiply them. 

Lastly we don't compute P(B). We skip this step because we don't actually need the exact probabilty. So we don't compute the exact probabilty but the number we compute will be proportional to the probabilty. We will compute this "pseudo probabilty" for each class. Our prediction will be the class with the highest probabilty.

In [8]:
prob_dict = {}
classes = np.unique(data["species"])
num_classes = len(classes)
num_features = len(X.columns)
num_samples = len(X)

for specie in classes: 
    prob_dict[specie] = {}
    prob_dict[specie]["feature"] = np.ones((num_classes,num_features))
    prob_dict[specie]["class_prob"] = 0


In [9]:
prob_dict

{0: {'feature': array([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
  'class_prob': 0},
 1: {'feature': array([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
  'class_prob': 0},
 2: {'feature': array([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
  'class_prob': 0}}

Here we have formed a probabilty dictionary which will hold probabilities. Each key (0,1,2) of the dictionary is a class. Each class has feature and class probabilities. Class probabilty is P(A) which is number of samples with label A divided by total number of samples. Feature probabilities are shown in a matrix. In this matrix columns represent features. So there are 4 features. Rows represent the value of the features. To better understand we will give and example. Lets say that there are 2 samples with feature #1 having value 0, 1 samples of feature #2 having value 3, 0 samples with feature #3 and 5 samples with feature #4 having value 3. Lets say that all of these samples have label 0 and we don't have any other samples with label 0. Our frequenct matrix for this class will be [[2,0,0,0],[0,0,0,0],[0,1,0,5]]. Notice that there are many zero entries. When we compute probabilities there will might be features with zero probabilty. This will be a problem because when we multiply the probabilities the result will be zero. To solve this problem we just add 1 to each case meaning we will assume there is at least one sample for each case. Thats why we initialize our matrix with ones instead of zeros. 

In [10]:
for x,label in zip(X.values,y.values):
    for feature,feature_val in enumerate(x):
        prob_dict[label]["feature"][feature_val,feature]+=1
    prob_dict[label]["class_prob"] += 1


Now we go all over the dataset and count the frequencies. In the outer loop we go over each sample. In the inner loop we go over each feature and increase the frequency.

In [11]:
prob_dict

{0: {'feature': array([[46.,  2., 51., 51.],
         [ 6., 35.,  1.,  1.],
         [ 1., 16.,  1.,  1.]]),
  'class_prob': 50},
 1: {'feature': array([[ 7., 22.,  1.,  1.],
         [39., 30., 49., 49.],
         [ 7.,  1.,  3.,  3.]]),
  'class_prob': 50},
 2: {'feature': array([[ 2., 12.,  1.,  1.],
         [28., 37.,  7.,  5.],
         [23.,  4., 45., 47.]]),
  'class_prob': 50}}

Lets look at feature matrix of class 0. From this matrix we can say that there are 45 examples with feature #1 having value 0. Feature #3 and #4 is always 0 in this class. You can see that because number of samples having these value is equal to total number of samples with label 0. (Don't forget that we added 1 to solve our zero probabilty problem). Now to compute probabilties we will divide our feature matrix with class_prob which is currently the number of samples belonging to that class. We will also compute P(A) that is class_prob. To calculate it we just have to divide class_prob with the total number of samples.

In [12]:
for i in prob_dict.keys():
   
    prob_dict[i]["feature"]/= (prob_dict[i]["class_prob"]+1) # +1 is because of the 1 we added in the beginning
    
    prob_dict[i]["class_prob"]/= num_samples
    prob_dict[i]["class_prob"] = prob_dict[i]["class_prob"]

In [13]:
prob_dict

{0: {'feature': array([[0.90196078, 0.03921569, 1.        , 1.        ],
         [0.11764706, 0.68627451, 0.01960784, 0.01960784],
         [0.01960784, 0.31372549, 0.01960784, 0.01960784]]),
  'class_prob': 0.3333333333333333},
 1: {'feature': array([[0.1372549 , 0.43137255, 0.01960784, 0.01960784],
         [0.76470588, 0.58823529, 0.96078431, 0.96078431],
         [0.1372549 , 0.01960784, 0.05882353, 0.05882353]]),
  'class_prob': 0.3333333333333333},
 2: {'feature': array([[0.03921569, 0.23529412, 0.01960784, 0.01960784],
         [0.54901961, 0.7254902 , 0.1372549 , 0.09803922],
         [0.45098039, 0.07843137, 0.88235294, 0.92156863]]),
  'class_prob': 0.3333333333333333}}

Now we can do predictions. As we said in the beginning we will multiply each probabilty. We will initilaize a probabilty matrix and write probabilities into this matrix. Here prob_dict[j]["feature"][X.iloc[i,:].values,np.arange(num_features)] will select each probabilty depending on the each value of the samples features.

In [14]:

prob_matrix = np.zeros((num_samples,num_classes))

for i in range(num_samples):
    spec = y[i]
    for j in classes:
        prob = prob_dict[j]["class_prob"] * prob_dict[j]["feature"][X.iloc[i,:].values,np.arange(num_features)].prod()
        prob_matrix[i,j] = prob
   

Lets look at the first 10 examples probabilites. 

In [15]:
prob_matrix[0:10]

array([[2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [9.43226964e-02, 3.44902292e-07, 3.94174048e-07],
       [9.43226964e-02, 3.44902292e-07, 3.94174048e-07],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06],
       [2.06330898e-01, 1.03470688e-05, 3.64610995e-06]])

As you can see our probabilities don't make up to 1. This is because we are not calculating exact probabilities as we have said before. To find the label we will take the argument with the highest value using argmax.

In [16]:
arg_max = np.argmax(prob_matrix,axis=1)

In [17]:
arg_max

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Now lets use sci-kit's Naive Bayes Classifier and compare with our implementations results.

In [18]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()

In [19]:
clf.fit(Xt, y.values)

CategoricalNB()

In [20]:
clf.predict_proba(Xt)[0:10]

array([[9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99992164e-01, 3.65659171e-06, 4.17896196e-06],
       [9.99992164e-01, 3.65659171e-06, 4.17896196e-06],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05],
       [9.99932185e-01, 5.01445357e-05, 1.76699792e-05]])

In [21]:
pred = clf.predict(Xt)
pred = pred.astype(np.int)
print(pred) 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Lets check what is the percentage of our predictions which are same with sci-kit's.

In [22]:
100*(pred == arg_max).sum()/num_samples

100.0

In [23]:
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [24]:
arg_max

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Lets also check our accuracy.

In [25]:
100*(y.values == pred).sum()/num_samples

96.0

We have %96 accuracy. Note that we did not divide our data into test,train sets as we should. We skipped these steps because we only wanted to implement the algorithm.